In [9]:
import sys
import os
import re

def main():
    file_to_read= 'amazon_cells_labelled.txt'
    positives=[] #list for positive sentences
    negatives=[] #list for negative sentences
    positive_counter=0 #keep track of number of positive sentences
    negative_counter=0 #keep track of number of negative sentences
    
     #opening file
    with open (file_to_read) as file:
        text=file.read()
        words = sorted(set(re.split(r'\W+', text)))
        words=[word.lower()for word in words]
        length=len(words)
        print("Length of vocabulary set is:", length)
    file.close()

NameError: name 'file_to_read' is not defined